In [1]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
from utils.point_cloud_util import load_labels, colorize_point_cloud
import multiprocessing

In [2]:
sparse_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/sparse"
dense_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/dense"
gt_dir = "/home/ylao/data/semantic3d"

In [3]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

# Sparse labels
sparse_labels = np.array(load_labels(os.path.join(sparse_dir, file_prefix + ".labels")), dtype=np.int32)

# Sparse points
sparse_pcd = open3d.read_point_cloud(os.path.join(sparse_dir, file_prefix + ".pcd"))
sparse_points = np.asarray(sparse_pcd.points)

# Dense points
dense_pcd = open3d.read_point_cloud(os.path.join(gt_dir, file_prefix + ".pcd"))
dense_points = np.asarray(dense_pcd.points)

# Ground-trugh dense labels
gt_labels = load_labels(os.path.join(gt_dir, file_prefix + ".labels"))

In [4]:
# Build KNN tree
sparse_pcd_tree = open3d.KDTreeFlann(sparse_pcd)

In [8]:
def match_knn_label(dense_index):
    global dense_points
    global sparse_labels
    global sparse_pcd_tree
    
    dense_point = dense_points[dense_index]
    k, sparse_indexes, _ = sparse_pcd_tree.search_hybrid_vector_3d(dense_point, 0.2, 20)
    if k == 0:
        k, sparse_indexes, _ = sparse_pcd_tree.search_knn_vector_3d(dense_point, 20)
    knn_sparse_labels = sparse_labels[sparse_indexes]
    dense_label = np.bincount(knn_sparse_labels).argmax()
        
    return dense_label

# Assign labels
start = time.time()
dense_indexes = list(range(len(dense_points)))
with multiprocessing.Pool() as pool:
    dense_labels = pool.map(match_knn_label, dense_indexes)
print("knn match time: ", time.time() - start)

# Eval
cm = ConfusionMatrix(9)
cm.increment_from_list(gt_labels, dense_labels)
cm.print_metrics()

knn match time:  44.59255909919739
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14477      74    1838     996    9113      70   52799     152 
          1       0 13152560    7151     769   33660  456277    6905    1746    4005 
          2       0   21121    3057     773    5048     970    1630     116     241 
          3       0    2980     299  216526   30403   24112     984     930      83 
          4       0    7600   10832  199479  898949   86461  106719     392   15620 
          5       0  411419     890   22573    2729 8272524   22027   12763   11016 
          6       0  136155    1314   15429  157891 2738526  335954  105994    8474 
          7       0   13729       0     370     279     236     242   71558      68 
          8       0   17830       0       0     835   29199    1435   23360  184693 
IoU per class:
[0.9214407800190936,
 0.05720220051644774,
 0.41985999883655545,
 0.5773978625432

## r = infinity, k = 10

```
knn match time:  44.59255909919739
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14477      74    1838     996    9113      70   52799     152 
          1       0 13152560    7151     769   33660  456277    6905    1746    4005 
          2       0   21121    3057     773    5048     970    1630     116     241 
          3       0    2980     299  216526   30403   24112     984     930      83 
          4       0    7600   10832  199479  898949   86461  106719     392   15620 
          5       0  411419     890   22573    2729 8272524   22027   12763   11016 
          6       0  136155    1314   15429  157891 2738526  335954  105994    8474 
          7       0   13729       0     370     279     236     242   71558      68 
          8       0   17830       0       0     835   29199    1435   23360  184693 
IoU per class:
[0.9214407800190936,
 0.05720220051644774,
 0.41985999883655545,
 0.5773978625432511,
 0.6841477169256786,
 0.09230319486965746,
 0.30872842270572043,
 0.6221573204787458]
mIoU (ignoring label 0):
0.46040468711189375
Overall accuracy
0.8293030194735018

```

## r = infinity, k = 100

```
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14882      74    1815     984    9073      94   52478     119 
          1       0 13166522    7464     856   36445  439396    6710    1490    4190 
          2       0   20971    2983     822    5186     985    1670     116     223 
          3       0    2866      31  215306   31065   25354     970     611     114 
          4       0    6758    9212  202284  916373   82060   96743      75   12547 
          5       0  393901     644   21655    1423 8296483   22662    8628   10545 
          6       0   84200    1235   15695  160148 2774004  345031  110813    8611 
          7       0   14662      23     363     337     179     276   70578      64 
          8       0   17951       0       0     745   21105     390   16467  200694 
IoU per class:
[0.9269338152127984,
 0.05784931639678076,
 0.4156550680319387,
 0.5868915160167055,
 0.6857150626364573,
 0.09507191475267816,
 0.3141239618661041,
 0.6834555893831348]
mIoU (ignoring label 0):
0.4707120305370748
Overall accuracy
0.8321042687427123
```

## r = 100, k = 100

```
knn match time:  57.63151741027832
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14882      74    1815     984    9075      92   52478     119 
          1       0 13166203    7464     854   36434  439739    6706    1498    4175 
          2       0   20958    3000     822    5181     985    1669     116     225 
          3       0    2865      29  215332   31045   25357     967     609     113 
          4       0    6760    9219  202259  916578   82093   96413      72   12658 
          5       0  394261     645   21689    1409 8295959   22686    8733   10559 
          6       0   83435    1240   15693  160055 2775520  344372  110791    8631 
          7       0   14666      23     363     337     180     274   70575      64 
          8       0   17957       0       0     743   21076     351   16264  200961 
IoU per class:
[0.9269379170267478,
 0.05816658911121452,
 0.41570124923503415,
 0.5870773274850505,
 0.6855659636963922,
 0.09489961290265689,
 0.31427426357624744,
 0.6840596779189657]
mIoU (ignoring label 0):
0.4708353251190387
Overall accuracy
0.8320687822134346
```

## r = 0.1, k = 100
```
knn match time:  49.399659633636475
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14861      73    1814     985    9095      94   52478     119 
          1       0 13129169    7304     798   35057  478075    6929    1624    4117 
          2       0   20960    3078     768    5170     992    1615     125     248 
          3       0    2898      72  215055   30898   25309     977     987     121 
          4       0    7287    9830  200592  909468   83278  102616      82   12899 
          5       0  391447     812   22236    2311 8291716   22830   13299   11290 
          6       0  100604    1247   15249  159068 2753117  352754  109412    8286 
          7       0   12925      13     369     300     225     261   72322      67 
          8       0   17822       0       0     794   21755     777   17983  198221 
IoU per class:
[0.9234827477158357,
 0.058927135582187845,
 0.41650769180115776,
 0.5831231366011606,
 0.684208823856568,
 0.09702393624190055,
 0.3144516813482091,
 0.6733507711121679]
mIoU (ignoring label 0):
0.46888449053239845
Overall accuracy
0.8305920766107568
```

## r = 0.1, k = 20

```
knn match time:  59.716012954711914
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14509      75    1825     995    9092      88   52785     150 
          1       0 13139264    7186     793   34435  468846    6857    1678    4014 
          2       0   20948    3051     784    5177     984    1642     123     247 
          3       0    2963     114  215540   30742   24887     993     965     113 
          4       0    7438   10099  200040  904850   85840  103490     179   14116 
          5       0  401101     831   22492    2378 8282403   22505   12933   11298 
          6       0  110376    1297   15284  156764 2753573  345390  108534    8519 
          7       0   13072       1     377     304     190     221   72250      67 
          8       0   18108       0       0     789   23866    1332   20171  193086 
IoU per class:
[0.9228904327917264,
 0.05813200213398369,
 0.41764276178241266,
 0.5812836742704323,
 0.6836978842965737,
 0.09497145411995292,
 0.3126825784952286,
 0.6529219615454847]
mIoU (ignoring label 0):
0.4655278436794744
Overall accuracy
0.8300203850395962
```

## r = 0.2, k = 10

```
knn match time:  47.98038911819458
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14480      74    1838     996    9114      70   52799     148 
          1       0 13144073    7128     772   33620  464771    6950    1839    3920 
          2       0   21123    3065     782    5041     962    1633     109     241 
          3       0    3016     301  216849   30210   23994     972     893      82 
          4       0    7604   10824  199230  898284   87374  106300     369   16067 
          5       0  413118     881   22781    2707 8270039   22287   12860   11268 
          6       0  130877    1324   15421  154497 2744654  337610  106725    8629 
          7       0   13592       0     370     279     220     241   71712      68 
          8       0   17935       0       0     824   28728    1444   22900  185521 
IoU per class:
[0.9210765014816047,
 0.057381959785823944,
 0.4205164901012851,
 0.5783328933899036,
 0.6830992139522728,
 0.0927611109462562,
 0.30886780344306286,
 0.6233339045180712]
mIoU (ignoring label 0):
0.46067123470228505
Overall accuracy
0.8289923151949375
```

## r = 0.1, k = 10 

```
knn match time:  46.46926927566528
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14479      74    1839     995    9113      70   52799     150 
          1       0 13144262    7132     768   33640  464577    6930    1837    3927 
          2       0   21123    3051     790    5037     965    1640     110     240 
          3       0    3017     305  216831   30226   23990     973     893      82 
          4       0    7612   10859  199210  898252   87390  106303     358   16068 
          5       0  413004     907   22801    2726 8270006   22277   12880   11340 
          6       0  131017    1330   15411  154632 2744521  337516  106714    8596 
          7       0   13582       0     370     285     220     241   71716      68 
          8       0   17930       0       0     826   28731    1447   22906  185512 
IoU per class:
[0.9210884548236281,
 0.057039765185365215,
 0.4204864767378948,
 0.5782400683908578,
 0.6831139234128247,
 0.09273569135508036,
 0.30888104057197,
 0.6232073449725034]
mIoU (ignoring label 0):
0.46059909568126556
Overall accuracy
0.8289920642800841
```